# MLflow workflow with remote tracking server (DagsHub)

In [9]:
import mlflow

**The following notebook uses DagsHub as remote tracking server, please note that you may not be able to run the scripts due to the demonstrated tracking server is private repository**

**If you want to run the scripts, you may follow these two notebooks from scratch**

https://colab.research.google.com/drive/1xeoZku-rQDtVkLGy8RNeITvrA-Y92OvN#scrollTo=dIO-Angr0vSJ
https://colab.research.google.com/drive/1lLO8CmqJ2wLY5nTHNMvvMYwHEJQtqQNo?usp=sharing


### MLflow UI

mlflow ui link: https://dagshub.com/jadon.ng/mario_vs_wario.mlflow (noted that you need to be logged in to dagshub (assuming you have access to the repository) to be able to access the ui)

## Setting up workspace

### Get DAGSHUB_TOKEN (optional, alternatively you can copy the token from DAGSHUB)

In [10]:
DAGSHUB_USER_NAME = "jadon.ng" 
DAGSHUB_EMAIL = "jadon.ng@carnot-innovations.com" 

In [11]:
# Get DAGSHUB_TOKEN
import requests
import getpass
import datetime

r = requests.post('https://dagshub.com/api/v1/user/tokens',
                  json={"name": f"colab-token-{datetime.datetime.now()}"},
                  auth=(DAGSHUB_USER_NAME, getpass.getpass('DAGsHub password:')))
r.raise_for_status()
DAGSHUB_TOKEN=r.json()['sha1']

DAGsHub password: ········


### Set up important variables 

To ensure that only authenticate collaborators will run/commit on the dagshub repository, you need to set up the identification variables to gain access.

In [12]:
import os

os.environ['MLFLOW_TRACKING_URI']="https://dagshub.com/jadon.ng/mario_vs_wario.mlflow" # the tracking uri

os.environ['MLFLOW_TRACKING_USERNAME'] = DAGSHUB_USER_NAME
os.environ['MLFLOW_TRACKING_PASSWORD'] = DAGSHUB_TOKEN

**Set tracking uri to connect to the mlflow tracking server**

In [13]:
mlflow.set_tracking_uri(os.environ['MLFLOW_TRACKING_URI'])

Check that we have successfully establish the connection by printing out all the registered models on the tracking server

In [14]:
client = mlflow.MlflowClient()

# loop through all registered models
# `"name LIKE '%'"` will match all model names <- SQL query
for model in client.search_registered_models(filter_string="name LIKE '%'"):
    # loop through the latest versions for each stage of a registered model
    for model_version in model.latest_versions:
        print(f"name={model_version.name}; run_id={model_version.run_id}; version={model_version.version}, stage={model_version.current_stage}")

name=Cool Mario Model; run_id=84814069b039487c81d02ca382f96160; version=1, stage=None
name=Great Mario Model; run_id=71cacc0abf404ad5af8d9749bf18da9d; version=1, stage=Staging
name=iris_model; run_id=790e26ade93b479ab7984a2d0eddac74; version=1, stage=Staging
name=Wicked Wario Model; run_id=daf93da97e0e4a5da34ad9957626b30b; version=1, stage=None


### Set up tracking server on terminal/command prompt

Mac/Linux/Unix:

`export MLFLOW_TRACKING_URI=<tracking-uri>`  <br>

Windows:

`set MLFLOW_TRACKING_URI=<tracking-uri>`

## Load and use models

In [ ]:
# make sure you set up tracking uri to establish connection to mlflow remote tracking server if you haven't
mlflow.set_tracking_uri("https://dagshub.com/jadon.ng/mario_vs_wario.mlflow") # the tracking uri 

In [ ]:
# Load model from registered model
model_uri = "models:/Wicked Wario Model/1" # models:/<model name>/<version>
model = mlflow.pyfunc.load_model(model_uri) 

The problem with the above method is that if you are logging the model on a different server/device than the device you are trying to load it on, there may be problems relating to the environment setting that make it unable to run (eg. if the computer you are using to load the model doesn't have tensorflow install/using different version of python, it may result in error)

## Example - Model Registry and Serving

### Create a very simple ML model

In [15]:
import mlflow
from mlflow.models import infer_signature

import pandas as pd
from sklearn import datasets
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score

In [31]:
mlflow.set_tracking_uri(uri="https://dagshub.com/jadon.ng/mario_vs_wario.mlflow")

In [33]:
# Load the Iris dataset
X, y = datasets.load_iris(return_X_y=True)

# Split the data into training and test sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Define the model hyperparameters
params = {"solver": "lbfgs", "max_iter": 1000, "multi_class": "auto", "random_state": 8888}

# Train the model
lr = LogisticRegression(**params)
lr.fit(X_train, y_train)

# Predict on the test set
y_pred = lr.predict(X_test)

# Calculate accuracy as a target loss metric
accuracy = accuracy_score(y_test, y_pred)

In [34]:
mlflow.set_experiment("Example Experiment")

2024/01/04 12:49:41 INFO mlflow.tracking.fluent: Experiment with name 'Example Experiment' does not exist. Creating a new experiment.


<Experiment: artifact_location='mlflow-artifacts:/5dcf012e9b0d465e88c5a6683e69df66', creation_time=1704343781865, experiment_id='1', last_update_time=1704343781865, lifecycle_stage='active', name='Example Experiment', tags={}>

### Log it on MLflow

In [35]:
import os
# Start an MLflow run
with mlflow.start_run(run_name='run1'):
    # Log the hyperparameters
    mlflow.log_params(params)

    # Log the loss metric
    mlflow.log_metric("accuracy", accuracy)

    # Set a tag that we can use to remind ourselves what this run was for
    mlflow.set_tag("Training Info", "Basic LR model for iris data")

    # Infer the model signature
    signature = infer_signature(X_train, lr.predict(X_train))
    
    # Log the model
    model_info = mlflow.sklearn.log_model(
        sk_model=lr,
        artifact_path='iris_model',
        signature=signature,
        input_example=X_train,
        registered_model_name="iris_model", # register the model directly with log_model function
    )

C:\Users\carnot\anaconda3\envs\mlflow_testing\Lib\site-packages\_distutils_hack\__init__.py:33: UserWarning: Setuptools is replacing distutils.
  warnings.warn("Setuptools is replacing distutils.")
Successfully registered model 'iris_model'.
2024/01/04 12:55:53 INFO mlflow.tracking._model_registry.client: Waiting up to 300 seconds for model version to finish creation. Model name: iris_model, version 1
Created version '1' of model 'iris_model'.


Alternatively, you can register the model seperately, see more here https://mlflow.org/docs/latest/model-registry.html

### Transition model version stage to Production

You can update your model stages using mlflow API, alternatively, you can use the mlflow ui to change it manually in the 'Models' tab on top -> select the respective model -> update the stage

In [ ]:
# Change the stage of the model to 'Production'
model_name = 'iris_model'
model_version = 1

client = mlflow.tracking.MlflowClient()
client.transition_model_version_stage(
    name=model_name,
    version=model_version,
    stage='Production'
)

### Load the model and test it

In [66]:
model_uri = "models:/iris_model/1" # models:/<model name>/<version>
model_uri = "models:/iris_model/staging" # alternative instead of version
model = mlflow.pyfunc.load_model(model_uri) 

In [39]:
predictions = model.predict(X_test)

In [40]:
predictions

array([1, 0, 2, 1, 1, 0, 1, 2, 1, 1, 2, 0, 0, 0, 0, 1, 2, 1, 1, 2, 0, 2,
       0, 2, 2, 2, 2, 2, 0, 0])

### Serving the model locally with CLI

https://mlflow.org/docs/latest/getting-started/quickstart-2/index.html#serve-the-model-locally

**Make sure you declared the tracking uri on your terminal first**

For Mac/Linux/Unix

`export MLFLOW_TRACKING_URI="https://dagshub.com/jadon.ng/mario_vs_wario.mlflow"`

For Windows
`set MLFLOW_TRACKING_URI="https://dagshub.com/jadon.ng/mario_vs_wario.mlflow"`

**Serve the model**
`mlflow models serve -m "models:/iris_model/1" --port 5002`

**Access the model with REST API through curl command** <br>
`curl http://127.0.0.1:5002/invocations -H 'Content-Type: application/json' -d '{ "dataframe_split": { "data": [[6.1, 2.8, 4.7, 1.2], [5.7, 3.8, 1.7, 0.3]] } }'`

Alternatively you can get the request on Python, below is an example with image as input

https://blog.devgenius.io/mlflow-tutorial-ml-tracking-and-serving-49abd929c8d9

In [ ]:
# Note that you still need to set tracking uri and serve the model on a specified port before running this
from pathlib import Path
import requests
import os
from torch.nn import functional as F
from image_transform import ImageTransform
from PIL import Image

base_dir = Path(__file__).absolute().parent.parent
data_dir = base_dir.joinpath('data')
test_folder = data_dir.joinpath('test')
image = os.listdir(test_folder)[0]

if __name__ == "__main__":
    endpoint = "http://localhost:5002/invocations"
    data = {
        'instances': str(test_folder.joinpath(image))
    }
    # do a post request
    response = requests.post(endpoint, json=data)
    print(response.json())

### Build a docker container image for your model

https://mlflow.org/docs/latest/getting-started/quickstart-2/index.html#build-a-container-image-for-your-model

**Build the docker image**

`mlflow models build-docker --model-uri "models:/iris_model/1" --name "iris"`

**Run the docker**

`docker run -p 5002:8080 iris`

**Use the model, basically same method with local serving, remember to input the right port**

`curl http://127.0.0.1:8080/invocations -H 'Content-Type: application/json' -d '{ "dataframe_split": { "data": [[6.1, 2.8, 4.7, 1.2], [5.7, 3.8, 1.7, 0.3]] } }'`